In [1]:
%pip install gtfs_kit
%pip install h3
%pip install keplergl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
from pathlib import Path
import json
import h3

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

import gtfs_kit as gk

In [3]:
path = "../resources/rejseplanen.zip"
feed = gk.read_feed(path, dist_units='km')

dates = ['20220320', '20220321', '20220322', '20220323', '20220324', '20220325', '20220326']

In [4]:
stops = feed.stops

stops['h3'] = stops.apply(lambda row: h3.geo_to_h3(row['stop_lat'],row['stop_lon'],10), axis=1)
stops.head()

stop_h3_map = pd.Series(stops.h3.values,index=stops.stop_id).to_dict()

In [5]:
ts = feed.compute_stop_time_series(dates=dates, freq='60Min')
ts.shape

(168, 38276)

In [6]:
tts = ts.transpose().reset_index(level=1)
tts.head()

datetime,stop_id,2022-03-20 00:00:00,2022-03-20 01:00:00,2022-03-20 02:00:00,2022-03-20 03:00:00,2022-03-20 04:00:00,2022-03-20 05:00:00,2022-03-20 06:00:00,2022-03-20 07:00:00,2022-03-20 08:00:00,...,2022-03-26 14:00:00,2022-03-26 15:00:00,2022-03-26 16:00:00,2022-03-26 17:00:00,2022-03-26 18:00:00,2022-03-26 19:00:00,2022-03-26 20:00:00,2022-03-26 21:00:00,2022-03-26 22:00:00,2022-03-26 23:00:00
indicator,,,,,,,,,,,,,,,,,,,,,
num_trips,000000000006,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,...,6.0,6.0,5.0,3.0,3.0,3.0,3.0,3.0,3.0,2.0
num_trips,000000000008,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,...,6.0,6.0,5.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0
num_trips,000000000009,2.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0,...,6.0,6.0,5.0,3.0,3.0,3.0,3.0,2.0,3.0,3.0
num_trips,000000000011,3.0,1.0,1.0,1.0,1.0,2.0,4.0,8.0,8.0,...,17.0,17.0,16.0,10.0,9.0,9.0,8.0,9.0,8.0,8.0
num_trips,000000000012,2.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,5.0,...,9.0,9.0,8.0,6.0,6.0,6.0,5.0,6.0,5.0,5.0


In [7]:
def add_lists(lists):
    res = np.array([0]*len(lists[0]))
    for i in range(len(lists)-1):
        l = np.array(lists[i])
        l = np.nan_to_num(l)
        # print(f"adding res with {np.shape(res)} and list with {np.shape(l)}")
        res = np.add(res, l)
        # print(f"resulting shape: {np.shape(res)}")
    return res

In [8]:
result_df = pd.DataFrame()
# result_df['h3'] = tts.apply(lambda row: stop_h3_map[row['stop_id']], axis=1)
result_df['stop_id'] = tts['stop_id'].values.tolist()
result_df['h3'] = result_df.apply(lambda row: stop_h3_map[row['stop_id']], axis=1)
result_df['frequency'] = tts.iloc[:,1:].values.tolist()

unique_h3s = set(stop_h3_map.values())
h3_map = {}
for index in unique_h3s:
    res = result_df.loc[result_df['h3'] == index]
    if res.shape[0] > 1:
        res = add_lists(res['frequency'].values.tolist())
        h3_map[index] = res.tolist()
    elif res['frequency'].any():
        h3_map[index] = np.nan_to_num(res['frequency'].values[0]).tolist()
    

In [9]:
new = pd.DataFrame()
new['h3'] = h3_map.keys()
new['freq'] = h3_map.values()
new['mode'] = ...
new.head()

,h3,freq
0,8a1f232d22dffff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,8a1f268a8357fff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,8a1f232d3d9ffff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,8a1f22836177fff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,8a1f05d55b67fff,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, ..."


In [10]:
time_cols = tts.columns[1:]

In [11]:
insane_array = []
for key, value in h3_map.items():
    for i, v in enumerate(value):
        insane_array.append([key, str(time_cols[i]), v])

insane_df = pd.DataFrame(insane_array, columns=['hex_id', 'time', 'height'])

In [12]:
from keplergl import KeplerGl
map = KeplerGl(height=500, data={'data': insane_df})
map.save_to_html()

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to keplergl_map.html!
